In [2]:
import nibabel as nib
import numpy as np
import os, random
import matplotlib.pyplot as plt
import scipy.ndimage
from skimage import morphology
from skimage import measure
from skimage.transform import resize
from scipy.ndimage.interpolation import rotate
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from skimage import feature

In [3]:
def resample(image, spacing, new_spacing):

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor)
    
    return image

def normalize_data(scan):

    scan = scan.astype('float32')
    scan -= np.mean(scan)
    scan /= np.max(scan)

    imax = np.max(scan)
    imin = np.min(scan)

    scan -= imin
    scan /= (imax-imin)
    
    return scan

def save(scan, output_path):
    np.save(output_path, scan)
    
    
def read_and_save(input_path):
    
    path = input_path.split('/')
    name = path[-1].split('.')[0]
    output_path = 'data/' + name
    
    scan = nib.load(input_path)
    
    scan_img = scan.get_data()
    header = scan.header
    
    slice_thickness = header.get_zooms()[2]
    pixel_dims = header.get_zooms()[0:2]
    
    spacing = np.array([slice_thickness, pixel_dims[0], pixel_dims[1]])
    scan = scan_img.transpose(2,0,1)
    
    scan = normalize_data(scan)
    scan = resample(scan, spacing, [1,1,1])
    
    save(scan, output_path)

In [7]:
files = os.listdir('../Nov_Normals_nii/')
files = [file for file in files if '.nii' in file]

In [ ]:
f